In [2]:
# #파이썬으로 gz 파일 압축 풀기
# import tarfile
# with tarfile.open(r'E:\Programming\python\aclImdb_v1\aclImdb_v1.tar.gz', 'r:gz') as tar:
#     tar.extractall()

In [3]:
import pyprind
import pandas as pd
import os

basepath = r'E:\Programming\python\aclImdb_v1\aclImdb'

##영화 리뷰를 한 줄씩 읽어서 데이터 프레임화 하기
labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:09:36


In [6]:
import numpy as np

#클래스 레이블 순서대로 데이터가 나열되있으므로 permutation 함수로 데이터 섞기
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv(r'E:\Programming\python\ML\data\movie_data.csv', index=False, encoding='utf-8')

In [7]:
#데이터 csv파일로 저장하기
df = pd.read_csv(r'E:\Programming\python\ML\data\movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
#데이터 프레임 행과 열의 개수 확인하기
df.shape

(50000, 2)

In [5]:
####Bag of Word####
#1. 전체 문서에 대해 고유한 토큰(예컨데, 단어로 이루어진 어휘 사전) 생성
#2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터 생성

In [6]:
###BoW 만들기
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #CountVectorizer은 문서 또는 문장으로 이루어진 텍스트 데이터 배열을 입력 받아 BoW 모델 생성

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [7]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [8]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [9]:
###tf(단어 빈도)-idf(역문서 빈도)###
#특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [11]:
##불필요한 문자 제거를 통한 텍스트 정제

In [12]:
#랜덤하게 섞은 데이터셋의 첫 번째 문서에서 마지막 50개의 문자를 출력
df.loc[0, 'review'][-50:] #HTML 마크업 문자와 글자가 아닌 문자 또한 포함되어있음

'and I suggest that you go see it before you judge.'

In [13]:
##이모티콘은 정보가 될수 있으므로 제외하고 그 외의 모든 구두점 문자 제거
#정규표현식 사용
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #html 마크업을 삭제하는 정규표현식
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [14]:
preprocessor(df.loc[0, 'review'][-50:])

'and i suggest that you go see it before you judge '

In [15]:
df['review'] = df['review'].apply(preprocessor)

In [16]:
##공백 문자를 기준으로 단어를 나누기
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [17]:
##nltke의 어간추출 알고리즘
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [18]:
##불용어##
#모든 텍스트에 흔하게 등장하는 단어
#tf-idf 보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 유용
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
#불용어 적용하기
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [20]:
##영화 리뷰를 긍정과 부정 리뷰로 분리하는 로지스틱 회귀 모델 훈련 시키기##
#1. 정제된 텍스트 문서가 저장된 dataframe을 2만 5천개는 훈련 세트, 2만 5천개는 테스트 세트로 나누기

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [22]:
#GridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 최적의 매개변수 조함 확인
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                   tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
              ]
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(solver='libinear', random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 14.3min


In [ ]:
print('최적의 매개변수 조합: %s ' % gs_lr_tdidf.best_params_)